# Importando as bibliotecas

In [3]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from collections import Counter
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados

# Dataset

In [4]:
procData = ProcessarDados("../dataset/norm_bin_10_FEATURES_M17_CM6b_TH199.csv")

# Funções

In [5]:
# funcção que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada
def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_f1 = []
    resultados_parametros = []
    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        # divisão os dados 
        X_train, X_test, y_train, y_test = procData.holdout(0.2)
        #print(Counter(y_test))

        # realizando o grid search para encontrar a melhor combinação entre o C, gamma e Kernel, 
        # considerando a acurácia (taxa de acerto)
        # aqui o método GridSearchCV é configurado para subdividir os dados de treino em k_folds
        
        
        clf = GradientBoostingClassifier(random_state=None)
        grid_gb = GridSearchCV(clf, param_grid, cv=k_folds, scoring='accuracy', verbose=0, n_jobs=-1)
        grid_gb.fit(X_train, y_train)

        # Treinando do modelo com os melhores parametros encontrados
        #n_estimators_best = grid_gb.best_estimator_.n_estimators
        #learning_rate_best = grid_gb.best_estimator_.learning_rate
        min_samples_best = grid_gb.best_estimator_.min_samples_leaf
        max_depth_best = grid_gb.best_estimator_.max_depth

        #GB = GradientBoostingClassifier(random_state = None, n_estimators = n_estimators_best, learning_rate =  learning_rate_best, max_depth = max_depth_best)
        
        GB = GradientBoostingClassifier(random_state = None, max_depth = max_depth_best, min_samples_leaf=min_samples_best)
        GB.fit(X_train, y_train)

        #testando o modelo
        y_pred = GB.predict(X_test)
        if exibir_matriz_confusao:
            print(confusion_matrix(y_test, y_pred))

        # calculado as metricas
        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred)
        f1_score = 2 * (precision * recall) / (precision + recall)

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_f1.append(f1_score)

        #best_parametros = "n_estimators: "+ str(n_estimators_best)+ " | learning_rate: "+ str(learning_rate_best)+ " | max_depth: "+ str(max_depth_best);
        best_parametros = " max_depth: "+ str(max_depth_best) +"| min_samples_leaf:"+ str(min_samples_best);
        resultados_parametros.append(best_parametros)


        if exibir_metricas:
            print("Rodada: #",i)
            print(best_parametros)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("F1-Score:",f1_score)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['f1'] = resultados_f1
    
    return resultados_gerais

In [6]:
def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False, salvarResultados=True):
        
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== TABELA DE MÉTRICAS DO MODELO ===================================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\t F1-Score")
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='    ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['f1'], axis=0), np.std(dict_metricas['f1'], axis=0)))
        print ("====================================================================================================")
        
    if(salvarResultados):
        # save to npy file
        np.save('../resultados/resultados_'+nome_modelo+'.npy', dict_metricas)

# Definindo os parâmetros

In [7]:
#n_estimators  = [100, 110, 120]
#learning_rate = [0.1, 0.3, 0.5, 1.0]
max_depth  = np.arange(2,9,2)
min_samples_leaf = np.arange(1,10,2)
#max_depth = [3]
#param_grid = {'n_estimators': n_estimators, 'learning_rate' : learning_rate, 'max_depth': max_depth}
param_grid = {'max_depth': max_depth, 'min_samples_leaf': min_samples_leaf}

epocas = 50
k_folds = 5
exibir_matriz_confusao = True
exibir_metricas = True
salvarResultados = True
rodadas=False

# Treinando e obtendo as métricas do modelo

In [8]:
# treinando o modelo
dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('GBDT',dict_metricas, rodadas, salvarResultados)

[[20 13]
 [ 1 86]]
Rodada: # 0
 max_depth: 2| min_samples_leaf:3
Accuracy: 0.8833333333333333
Precision: 0.8686868686868687
Recall: 0.9885057471264368
F1-Score: 0.9247311827956989


[[19  2]
 [ 0 99]]
Rodada: # 1
 max_depth: 4| min_samples_leaf:1
Accuracy: 0.9833333333333333
Precision: 0.9801980198019802
Recall: 1.0
F1-Score: 0.99


[[28 10]
 [ 0 82]]
Rodada: # 2
 max_depth: 6| min_samples_leaf:5
Accuracy: 0.9166666666666666
Precision: 0.8913043478260869
Recall: 1.0
F1-Score: 0.9425287356321839


[[23  5]
 [ 2 90]]
Rodada: # 3
 max_depth: 4| min_samples_leaf:7
Accuracy: 0.9416666666666667
Precision: 0.9473684210526315
Recall: 0.9782608695652174
F1-Score: 0.9625668449197862


[[22  5]
 [ 6 87]]
Rodada: # 4
 max_depth: 4| min_samples_leaf:1
Accuracy: 0.9083333333333333
Precision: 0.9456521739130435
Recall: 0.9354838709677419
F1-Score: 0.9405405405405404


[[21  5]
 [ 2 92]]
Rodada: # 5
 max_depth: 4| min_samples_leaf:9
Accuracy: 0.9416666666666667
Precision: 0.9484536082474226
Recall: 0.

[[18 12]
 [ 3 87]]
Rodada: # 47
 max_depth: 4| min_samples_leaf:3
Accuracy: 0.875
Precision: 0.8787878787878788
Recall: 0.9666666666666667
F1-Score: 0.9206349206349207


[[17  6]
 [ 3 94]]
Rodada: # 48
 max_depth: 4| min_samples_leaf:7
Accuracy: 0.925
Precision: 0.94
Recall: 0.9690721649484536
F1-Score: 0.9543147208121826


[[18  8]
 [ 2 92]]
Rodada: # 49
 max_depth: 4| min_samples_leaf:1
Accuracy: 0.9166666666666666
Precision: 0.92
Recall: 0.9787234042553191
F1-Score: 0.9484536082474226


============================================== GBDT =================================================
=================================== TABELA DE MÉTRICAS DO MODELO ===================================
	 Accuracy 	|	 Precision 	|	 Recall 	|	 F1-Score
      0.92 +- 0.02             0.93 +- 0.03          0.97 +- 0.02             0.95 +- 0.01
